In [5]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib_inline
import numpy as np
import pandas as pd
from tqdm import tqdm


from orc.primal import dobson, greedy, hall_hochbaum
from orc.relaxation import subgrad_opt, dual_lb
from orc.utils import generate_problem

mpl.rcParams["font.family"] = ["serif"]
mpl.rcParams["font.serif"] = ["cmr10"]
matplotlib_inline.backend_inline.set_matplotlib_formats("retina")

# Primal heuristics comparisons

## Without fixed variables

In [ ]:
table = []
funcs = [(greedy, "Greedy"), (dobson, "Dobson"),
         (hall_hochbaum, "Hall-Hochbaum")]
for rows, cols in tqdm([(5, 10), (10, 20), (20, 50), (50, 100)]):
    res = {"Rows": rows, "Cols": cols, "Greedy": 0, 
           "Dobson": 0, "Hall-Hochbaum": 0}
    for i in tqdm(range(10)):
        A, b = generate_problem(rows, cols)
        v = []
        for f, name in funcs:
            ub = np.sum(A, axis=0) @ f(A, b, [], [])
            v.append(ub)
        j = np.argmin(v)
        best = funcs[j][1]
        res[best] += 1
    table.append(res)

In [ ]:
table

In [ ]:
df = pd.DataFrame(table)
df = df.set_index(["Rows", "Cols"])
df.style.to_latex(column_format="llccc")

## With fixed variables

In [ ]:
np.random.seed(42)

table = []
funcs = [(greedy, "Greedy"), (dobson, "Dobson"),
         (hall_hochbaum, "Hall-Hochbaum")]
for rows, cols in tqdm([(5, 10), (10, 20), (20, 50), (50, 100)]):
    res = {"Rows": rows, "Cols": cols, "Greedy": 0, 
           "Dobson": 0, "Hall-Hochbaum": 0}
    for i in tqdm(range(10)):
        A, b = generate_problem(rows, cols)
        fixed_n = int(cols * 0.3) 
        fixed = np.random.choice(
            A.shape[-1], fixed_n, replace=False)
        x0 = fixed[:int(fixed_n / 2)]
        x1 = fixed[int(fixed_n / 2) + 1:]
        x = np.ones(A.shape[-1])
        x[x0] = 0
        if np.any(A @ x < b):
            continue
        v = []
        for f, name in funcs:
            ub = np.sum(A, axis=0) @ f(A, b, [], [])
            v.append(ub)
        j = np.argmin(v)
        best = funcs[j][1]
        res[best] += 1
    table.append(res)

In [ ]:
table

In [ ]:
df = pd.DataFrame(table)
df = df.set_index(["Rows", "Cols"])
df.style.to_latex(column_format="llccc")

# Subgradient optimization lower bounds

In [ ]:
data = {}
for rows, cols in tqdm([(10, 20), (20, 50), (40, 80), (80, 150)]):
    A, b = generate_problem(rows, cols)
    x = hall_hochbaum(A, b, [], [])
    ub = np.sum(A, axis=0) @ x

    res = []
    for omega in [10, 20, 40, 50, 70, 100, 200, 500]:
        lb = subgrad_opt(A, b, ub, [], [], omega=omega)
        res.append([omega, lb])
    data[(rows, cols)] = res

In [ ]:


fig, ax = plt.subplots()
for k, v in data.items():
    x = np.array(v)[:,0]
    y = np.array(v)[:,1]
    y = (y - y.min()) / (y.max() - y.min())
    ax.plot(x, y, label=f"{k[0]} rows, {k[1]} cols")
ax.legend()
ax.set_xlabel("Iterations")
ax.set_ylabel("Scaled lower bound")
ax.set_title("Subgradient optimization lower bounds")

# Subgradient optimization and LP relaxation of dual comparison

In [8]:
data = []
for i in range(20):
    A, b = generate_problem(20, 50)
    x = hall_hochbaum(A, b, [], [])
    ub = np.sum(A, axis=0) @ x
    lb1 = subgrad_opt(A, b, ub, [], [])
    lb2 = dual_lb(A, b, ub, [], [])
    data.append([lb1, lb2])

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i7-9700K CPU @ 3.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 50 rows, 20 columns and 500 nonzeros
Model fingerprint: 0xb990c72a
Coefficient statistics:
  Matrix range     [2e+01, 5e+02]
  Objective range  [2e+02, 2e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+03, 3e+03]
Presolve time: 0.00s
Presolved: 50 rows, 20 columns, 500 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.3158438e+33   3.098770e+32   1.315844e+03      0s
      28    2.6442140e+04   0.000000e+00   0.000000e+00      0s

Solved in 28 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.644213992e+04
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i7-9700K CPU @ 3.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 8 

In [9]:
data

[[26443.68011861691, 26442.139917169763],
 [23849.261305452816, 23291.904421364765],
 [26716.963261944096, 27953.999999999993],
 [24135.464306656882, 23171.999999999993],
 [24428.209684428806, 25207.48443792097],
 [26632.195947965596, 25041.3028289273],
 [24064.005721721787, 23557.360179848787],
 [32425.737783956316, 32740.50698729497],
 [27434.90982532425, 26997.59781944929],
 [24561.11044124887, 23747.993159957794],
 [27691.51198045919, 27052.700073975986],
 [28453.42140220441, 28073.78126187657],
 [24207.085639792065, 24337.0],
 [27231.203015732484, 27004.369881523126],
 [28511.640690393055, 28618.99999999999],
 [27885.40590972944, 27746.240732473107],
 [28115.378100354676, 27498.945404003745],
 [29088.592116289106, 28522.446922300136],
 [24429.789440322696, 24257.56625210902],
 [26463.358760932726, 25708.086817487525]]

2042.1219188768373

Set parameter Username
Academic license - for non-commercial use only - expires 2024-06-16
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i7-9700K CPU @ 3.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 20 rows, 10 columns and 100 nonzeros
Model fingerprint: 0xd66770a6
Coefficient statistics:
  Matrix range     [9e+00, 2e+02]
  Objective range  [6e+01, 4e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+01, 6e+02]
Presolve removed 1 rows and 0 columns
Presolve time: 0.00s
Presolved: 19 rows, 10 columns, 99 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.3167823e+03   2.060869e+02   0.000000e+00      0s
       9    2.0885909e+03   0.000000e+00   0.000000e+00      0s

Solved in 9 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.088590899e+03


2088.5908989086793